In [1]:
import sys
python_path = ['/home/chao/cantera/build/python', '/home/chao/RMG-Py', '/home/chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages', '/home/chao/cantera', '/home/chao/anaconda3/envs/rmg_env/lib/python37.zip', '/home/chao/anaconda3/envs/rmg_env/lib/python3.7', '/home/chao/anaconda3/envs/rmg_env/lib/python3.7/lib-dynload', '/home/chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/chemprop-0.0.1-py3.7.egg', '/home/chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/descriptastorus-2.0.0.32-py3.7.egg']
sys.path.clear()
for path in python_path:
    sys.path.append(path)

In [2]:
import cantera as ct
print(ct.__file__)

import yaml
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import rmgpy
import rmgpy.data.base
import rmgpy.molecule
import rmgpy.quantity

/home/chao/cantera/build/python/cantera/__init__.py


In [3]:
path_to_cti = '/home/chao/sketches/cpox_sim/cpox_no_cov_modified/cpox_no_cov_modified.yaml'
gas = ct.Solution(path_to_cti, 'gas')
surf = ct.Interface(path_to_cti, 'surface1', [gas])
species_dict = rmgpy.data.base.Database().get_species('/home/chao/sketches/cpox_sim/cpox_no_cov/species_dictionary.txt',resonance=False)

In [4]:
energy_c = np.arange(-5.5, -7.75, -0.25)
energy_o = np.arange(-3.25, -5.5, -0.25)

In [5]:
def find_species_phase_index(species_name):
    """
    Return the phase object (gas or surface) and the index
    of the named species.
    """
    try:
        i = gas.species_index(species_name)
        return gas, i
    except ValueError:
        i = surf.species_index(species_name)
        return surf, i
    
def change_species_enthalpy(species_name, dH, temp):
    """
    Find the species by name and change it's enthlapy by dH (in J/kmol)
    """
    phase, index = find_species_phase_index(species_name)

    species = phase.species(index)
    print(f"Initial H({temp}) = {species.thermo.h(temp)/1e6:.1f} kJ/mol")
    dx = dH / ct.gas_constant  # 'dx' is in fact (delta H / R). Note that R in cantera is 8314.462 J/kmol
    assert isinstance(species.thermo, ct.NasaPoly2)
    # print(species.thermo.coeffs)
    perturbed_coeffs = species.thermo.coeffs.copy()
    perturbed_coeffs[6] += dx
    perturbed_coeffs[13] += dx
    
    species.thermo = ct.NasaPoly2(species.thermo.min_temp, species.thermo.max_temp, 
                            species.thermo.reference_pressure, perturbed_coeffs)
#     print(species.thermo.coeffs)
    phase.modify_species(index, species)
    print(f"Modified H({temp}) = {species.thermo.h(temp)/1e6:.1f} kJ/mol")
    
# def correct_binding_energy(target_surf, spec_index, center_atom, x, temperature):
#     """
#     Here applies linear scaling relationship to calculate binding energy (J/kmol) from one metal
#     to another
#     Args:
#         target_surf: binding energy dictionary of interest eg: Pt_111
#         spec_index: index of the species to change
#         center_atom: center atom of the species to change
#         x: bonds of center atoms of the species to change
#     """
#     pt_111 = {'H': (-2.75368,'eV/molecule'),
#               'C': (-7.02516,'eV/molecule'),
#               'N': (-4.63225,'eV/molecule'),
#               'O': (-3.81153,'eV/molecule'),
#              }
#     x_max = {'H': 1,
#              'C': 4,
#              'N': 3,
#              'O': 2,
#             }
#     deltaE = ((x_max[center_atom]-x) / x_max[center_atom]) * (target_surf[center_atom][0] * 9.6487e7 - pt_111[center_atom][0] * 9.6487e7)
#     print(f"Applying LSR correction to {surf.species(spec_index).name}:")
#     change_species_enthalpy(surf.species(spec_index).name, deltaE, temperature)

In [6]:
def correct_binding_energy(species, delta_atomic_adsoprtion_energies={}):
    """
    Changes the thermo of the provided species, by applying a linear scaling relation
    to correct the adsorption energy.

    :param species: The species to modify (an RMG Species object)
    :param delta_atomic_adsoprtion_energies: a dictionary of changes in atomic adsorption energies to apply.
        mapping for each element an RMG Quantity objects with .value_si giving a value in J/mol.
    :return: None
    """
    molecule = species.molecule[0]
    # only want/need to do one resonance structure
    surface_sites = []
    for atom in molecule.atoms:
        if atom.is_surface_site():
            surface_sites.append(atom)
    normalized_bonds = {'C': 0., 'O': 0., 'N': 0., 'H': 0.}
    max_bond_order = {'C': 4., 'O': 2., 'N': 3., 'H': 1.}
    for site in surface_sites:
        numbonds = len(site.bonds)
        if numbonds == 0:
            # vanDerWaals
            pass
        else:
            assert len(site.bonds) == 1, "Each surface site can only be bonded to 1 atom"
            bonded_atom = list(site.bonds.keys())[0]
            bond = site.bonds[bonded_atom]
            if bond.is_single():
                bond_order = 1.
            elif bond.is_double():
                bond_order = 2.
            elif bond.is_triple():
                bond_order = 3.
            elif bond.is_quadruple():
                bond_order = 4.
            else:
                raise NotImplementedError("Unsupported bond order {0} for binding energy "
                                          "correction.".format(bond.order))

            normalized_bonds[bonded_atom.symbol] += bond_order / max_bond_order[bonded_atom.symbol]


    # now edit the adsorptionThermo using LSR
    change_in_binding_energy = 0.0
    for element in delta_atomic_adsoprtion_energies.keys():
        change_in_binding_energy += delta_atomic_adsoprtion_energies[element].value_si *  normalized_bonds[element]
    print(molecule, change_in_binding_energy, normalized_bonds)
    if change_in_binding_energy != 0.0:
        print(f"Applying LSR correction to {species.label}:")
        change_species_enthalpy(species.label, change_in_binding_energy*1000, 300)

In [7]:
pt_111 = {'H': -2.75368,'C': -7.02516,'N': -4.63225,'O': -3.81153}
for i in energy_c:
    for j in energy_o:
        delta_atomic_adsoprtion_energies = {}
        delta_atomic_adsoprtion_energies['C'] = rmgpy.quantity.Energy(i -pt_111['C'], 'eV/molecule')
        delta_atomic_adsoprtion_energies['O'] = rmgpy.quantity.Energy(j - pt_111['O'], 'eV/molecule')
        for species in surf.species():
            rmg_spcs = species_dict[species.name]
            correct_binding_energy(rmg_spcs, delta_atomic_adsoprtion_energies)
        surf.write_yaml( f'cpox_nocov_c{i}_o{j}.yaml')
        

<Molecule "[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
<Molecule "[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 1.0}
<Molecule "O=C=O.[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
<Molecule "O=C=[Pt]"> 73577.79237894919 {'C': 0.5, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to COX(23):
Initial H(300) = -281.4 kJ/mol
Modified H(300) = -207.8 kJ/mol
<Molecule "C.[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
<Molecule "O=[Pt]"> 54179.41429692798 {'C': 0.0, 'O': 1.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to OX(25):
Initial H(300) = -140.5 kJ/mol
Modified H(300) = -86.3 kJ/mol
<Molecule "C=[Pt]"> 73577.79237894919 {'C': 0.5, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to CH2X(26):
Initial H(300) = 0.7 kJ/mol
Modified H(300) = 74.3 kJ/mol
<Molecule "C[Pt]"> 36788.896189474595 {'C': 0.25, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to CH3X(27):
Initial H(300) = -45.7 kJ/mol
Modified H(300) = -8.9 kJ/mol
<Molecule "C#[Pt]"> 110366.688568

<Molecule "[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
<Molecule "[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 1.0}
<Molecule "O=C=O.[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
<Molecule "O=C=[Pt]"> 73577.79237894919 {'C': 0.5, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to COX(23):
Initial H(300) = 233.7 kJ/mol
Modified H(300) = 307.3 kJ/mol
<Molecule "C.[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
<Molecule "O=[Pt]"> -114669.93483780033 {'C': 0.0, 'O': 1.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to OX(25):
Initial H(300) = -267.7 kJ/mol
Modified H(300) = -382.4 kJ/mol
<Molecule "C=[Pt]"> 73577.79237894919 {'C': 0.5, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to CH2X(26):
Initial H(300) = 515.8 kJ/mol
Modified H(300) = 589.3 kJ/mol
<Molecule "C[Pt]"> 36788.896189474595 {'C': 0.25, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to CH3X(27):
Initial H(300) = 211.8 kJ/mol
Modified H(300) = 248.6 kJ/mol
<Molecule "C#[Pt]"> 110366.6

<Molecule "[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
<Molecule "[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 1.0}
<Molecule "O=C=O.[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
<Molecule "O=C=[Pt]"> 61517.124583611454 {'C': 0.5, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to COX(23):
Initial H(300) = 688.4 kJ/mol
Modified H(300) = 749.9 kJ/mol
<Molecule "C.[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
<Molecule "O=[Pt]"> -66427.26365644939 {'C': 0.0, 'O': 1.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to OX(25):
Initial H(300) = -491.5 kJ/mol
Modified H(300) = -557.9 kJ/mol
<Molecule "C=[Pt]"> 61517.124583611454 {'C': 0.5, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to CH2X(26):
Initial H(300) = 970.5 kJ/mol
Modified H(300) = 1032.0 kJ/mol
<Molecule "C[Pt]"> 30758.562291805727 {'C': 0.25, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to CH3X(27):
Initial H(300) = 439.2 kJ/mol
Modified H(300) = 469.9 kJ/mol
<Molecule "C#[Pt]"> 92275.

<Molecule "[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
<Molecule "[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 1.0}
<Molecule "O=C=O.[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
<Molecule "O=C=[Pt]"> 49456.45678827372 {'C': 0.5, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to COX(23):
Initial H(300) = 1082.9 kJ/mol
Modified H(300) = 1132.3 kJ/mol
<Molecule "C.[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
<Molecule "O=[Pt]"> -18184.592475098438 {'C': 0.0, 'O': 1.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to OX(25):
Initial H(300) = -811.8 kJ/mol
Modified H(300) = -830.0 kJ/mol
<Molecule "C=[Pt]"> 49456.45678827372 {'C': 0.5, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to CH2X(26):
Initial H(300) = 1364.9 kJ/mol
Modified H(300) = 1414.4 kJ/mol
<Molecule "C[Pt]"> 24728.22839413686 {'C': 0.25, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to CH3X(27):
Initial H(300) = 636.4 kJ/mol
Modified H(300) = 661.1 kJ/mol
<Molecule "C#[Pt]"> 74184

<Molecule "[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
<Molecule "[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 1.0}
<Molecule "O=C=O.[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
<Molecule "O=C=[Pt]"> 37395.78899293598 {'C': 0.5, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to COX(23):
Initial H(300) = 1454.4 kJ/mol
Modified H(300) = 1491.8 kJ/mol
<Molecule "C.[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
<Molecule "O=[Pt]"> 5936.743115577035 {'C': 0.0, 'O': 1.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to OX(25):
Initial H(300) = -1198.5 kJ/mol
Modified H(300) = -1192.5 kJ/mol
<Molecule "C=[Pt]"> 37395.78899293598 {'C': 0.5, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to CH2X(26):
Initial H(300) = 1736.5 kJ/mol
Modified H(300) = 1773.9 kJ/mol
<Molecule "C[Pt]"> 18697.89449646799 {'C': 0.25, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to CH3X(27):
Initial H(300) = 822.1 kJ/mol
Modified H(300) = 840.8 kJ/mol
<Molecule "C#[Pt]"> 56093

...

<Molecule "[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
<Molecule "[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 1.0}
<Molecule "O=C=O.[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
<Molecule "O=C=[Pt]"> 25335.121197598244 {'C': 0.5, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to COX(23):
Initial H(300) = 1893.5 kJ/mol
Modified H(300) = 1918.8 kJ/mol
<Molecule "C.[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
<Molecule "O=[Pt]"> -114669.93483780033 {'C': 0.0, 'O': 1.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to OX(25):
Initial H(300) = -1790.8 kJ/mol
Modified H(300) = -1905.4 kJ/mol
<Molecule "C=[Pt]"> 25335.121197598244 {'C': 0.5, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to CH2X(26):
Initial H(300) = 2175.6 kJ/mol
Modified H(300) = 2200.9 kJ/mol
<Molecule "C[Pt]"> 12667.560598799122 {'C': 0.25, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to CH3X(27):
Initial H(300) = 1041.7 kJ/mol
Modified H(300) = 1054.4 kJ/mol
<Molecule "C#[Pt]"

<Molecule "[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
<Molecule "[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 1.0}
<Molecule "O=C=O.[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
<Molecule "O=C=[Pt]"> 13274.453402260508 {'C': 0.5, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to COX(23):
Initial H(300) = 2037.1 kJ/mol
Modified H(300) = 2050.4 kJ/mol
<Molecule "C.[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
<Molecule "O=[Pt]"> -114669.93483780033 {'C': 0.0, 'O': 1.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to OX(25):
Initial H(300) = -2171.5 kJ/mol
Modified H(300) = -2286.2 kJ/mol
<Molecule "C=[Pt]"> 13274.453402260508 {'C': 0.5, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to CH2X(26):
Initial H(300) = 2319.2 kJ/mol
Modified H(300) = 2332.4 kJ/mol
<Molecule "C[Pt]"> 6637.226701130254 {'C': 0.25, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to CH3X(27):
Initial H(300) = 1113.5 kJ/mol
Modified H(300) = 1120.1 kJ/mol
<Molecule "C#[Pt]">

<Molecule "[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
<Molecule "[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 1.0}
<Molecule "O=C=O.[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
<Molecule "O=C=[Pt]"> 1213.7856069227719 {'C': 0.5, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to COX(23):
Initial H(300) = 2070.9 kJ/mol
Modified H(300) = 2072.1 kJ/mol
<Molecule "C.[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
<Molecule "O=[Pt]"> -90548.59924712486 {'C': 0.0, 'O': 1.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to OX(25):
Initial H(300) = -2461.7 kJ/mol
Modified H(300) = -2552.3 kJ/mol
<Molecule "C=[Pt]"> 1213.7856069227719 {'C': 0.5, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to CH2X(26):
Initial H(300) = 2353.0 kJ/mol
Modified H(300) = 2354.2 kJ/mol
<Molecule "C[Pt]"> 606.8928034613859 {'C': 0.25, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to CH3X(27):
Initial H(300) = 1130.4 kJ/mol
Modified H(300) = 1131.0 kJ/mol
<Molecule "C#[Pt]"> 

<Molecule "[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
<Molecule "[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 1.0}
<Molecule "O=C=O.[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
<Molecule "O=C=[Pt]"> -10846.882188414966 {'C': 0.5, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to COX(23):
Initial H(300) = 2020.3 kJ/mol
Modified H(300) = 2009.5 kJ/mol
<Molecule "C.[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
<Molecule "O=[Pt]"> -66427.26365644939 {'C': 0.0, 'O': 1.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to OX(25):
Initial H(300) = -2776.0 kJ/mol
Modified H(300) = -2842.5 kJ/mol
<Molecule "C=[Pt]"> -10846.882188414966 {'C': 0.5, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to CH2X(26):
Initial H(300) = 2302.4 kJ/mol
Modified H(300) = 2291.6 kJ/mol
<Molecule "C[Pt]"> -5423.441094207483 {'C': 0.25, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to CH3X(27):
Initial H(300) = 1105.1 kJ/mol
Modified H(300) = 1099.7 kJ/mol
<Molecule "C#[Pt]

<Molecule "C#[Pt]"> -34361.32497562905 {'C': 0.75, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to CHX(28):
Initial H(300) = 3224.7 kJ/mol
Modified H(300) = 3190.3 kJ/mol
<Molecule "C~[Pt]"> -45815.099967505404 {'C': 1.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to CX(29):
Initial H(300) = 4387.0 kJ/mol
Modified H(300) = 4341.2 kJ/mol
<Molecule "[H][H].[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
<Molecule "O[Pt]"> -21152.964032886957 {'C': 0.0, 'O': 0.5, 'N': 0.0, 'H': 0.0}
Applying LSR correction to OHX(31):
Initial H(300) = -1642.8 kJ/mol
Modified H(300) = -1664.0 kJ/mol
<Molecule "O.[Pt]"> 0.0 {'C': 0.0, 'O': 0.0, 'N': 0.0, 'H': 0.0}
<Molecule "O=C[Pt]"> -11453.774991876351 {'C': 0.25, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to CHOX(33):
Initial H(300) = 859.1 kJ/mol
Modified H(300) = 847.6 kJ/mol
<Molecule "OC[Pt]"> -11453.774991876351 {'C': 0.25, 'O': 0.0, 'N': 0.0, 'H': 0.0}
Applying LSR correction to CH3OX(61):
Initial H(300) = 798.0 k

In [8]:
energy_c = np.arange(-5.5, -7.75, -0.25)
energy_o = np.arange(-3.25, -5.5, -0.25)
with open('/home/chao/sketches/cpox_sim/cpox_no_cov_modified/cpox_no_cov_modified.yaml', 'r') as f:
    model = yaml.load(f, Loader=yaml.FullLoader)
for i in energy_c:
    for j in energy_o:
        path_to_cti = f'/home/chao/sketches/cpox_sim/cpox_no_cov_modified_surfs/cpox_nocov_c{i}_o{j}.yaml'
        with open(path_to_cti, 'r') as f:
            model_to_change = yaml.load(f, Loader=yaml.FullLoader)

        model_to_change['elements'] = model['elements']

        with open(path_to_cti, 'w') as f:
            yaml.dump(model_to_change, f, default_flow_style=False)
            
        gas = ct.Solution(path_to_cti, 'gas')
        surf = ct.Interface(path_to_cti, 'surface1', [gas])